## Week 3 : Vector Search

## Q1. Getting the embeddings model

In [1]:
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import requests
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

In [2]:
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
vector_question = embedding_model.encode(user_question)

/home/cecile/Documents/llm-zoomcamp/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


**What's the first value of the resulting vector?**

In [5]:
vector_question[0]

0.07822264

*Answer: 0.07*

### Prepare the documents

In [6]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'

docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
filtered_documents = [d for d in documents if d["course"] == "machine-learning-zoomcamp"]
len(filtered_documents)

375

## Q2. Creating the embeddings

In [8]:
embeddings = []

for doc in filtered_documents:
    qa_text = f"{doc['question']} {doc['text']}"
    vector = embedding_model.encode(qa_text).tolist()
    embeddings.append(vector)

In [9]:
X = np.array(embeddings)
X.shape

(375, 768)

## Q3. Search

**What's the highest score in the results?**

In [10]:
v = vector_question
scores = X.dot(v)

In [11]:
scores.max()

0.6506573332655323

*Answer:* 0.65

### Vector search

In [12]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine

In [13]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [14]:
for doc in tqdm(ground_truth):
    question = doc['question']

    doc['question_vector'] = embedding_model.encode(question)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [15]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_engine.search(q['question_vector'], num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [16]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

**What did you get?**

In [17]:
hit_rate(relevance_total)

0.9398907103825137

*Answer:* 0.93

## Q5. Indexing with ElasticSearch

In [18]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [19]:
for doc in tqdm(filtered_documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qt)

  0%|          | 0/375 [00:00<?, ?it/s]

In [20]:
for doc in tqdm(filtered_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [21]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [22]:
result = elastic_search(user_question)

In [23]:
for i in range(0, 5):
    question = result[i]['question']
    id = result[i]['id']

    print(f"{i+1}: {id}: {question}")

1: ee58a693: The course has already started. Can I still join it?
2: c5cde96c: Running out of storage after building many docker images
3: ed8b300d: How to install Tensorflow in Ubuntu WSL2
4: 8006b496: How to run a script while a web-server is working?
5: 6ba259b1: I filled the form, but haven't received a confirmation email. Is it normal?


**What's the ID of the document with the highest score?**

*Answer:* ee58a693

## Q6. Hit-rate for Elasticsearch

In [24]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [25]:
hit_rate(relevance_total)

0.8169398907103825

**What's hitrate for our dataset for Elastic?**

*Answer:* 0.81